In [1]:
#input dataset name and read file
def loadData(data_name):
    dataset = []
    with open(data_name, 'r') as f:
        while(True):
            l = f.readline()
            if(not l): break
            dataset.append(l.replace('\n', '').split(','))
    return dataset

In [2]:
#delete all the elements in the item set that not satisfy the min_sup
def least_freq_check(fre_item_set,least_freq):
    for i in list(fre_item_set.keys()):
        if fre_item_set[i] < least_freq:
            fre_item_set.pop(i)
    return fre_item_set

In [3]:
#create k-value freq item sets
def get_next_fre_item_set(data_set, fre_item_set, can_item_len, least_freq):
    freq_items = [i for i in fre_item_set.keys()]
    next_fre_item_set = {}
    for i in range(len(freq_items) - 1):
        for j in range(i + 1, len(freq_items)):
            ele = comb_ele(freq_items[i],freq_items[j])
            temp_key = set(ele)
            item_list = []
            for key in next_fre_item_set.keys():
               item_list.append(set(key))
            if len(temp_key) <= can_item_len:
                if temp_key not in item_list:
                    for record in data_set:
                        if temp_key.issubset(set(record)):
                            data_list = []
                            for key in next_fre_item_set.keys():
                                data_list.append(set(key))
                            if temp_key in data_list:
                                next_fre_item_set[tuple(temp_key)] += 1
                            else:
                                next_fre_item_set[tuple(temp_key)] = 1

    
    next_fre_item_sets = least_freq_check(next_fre_item_set,least_freq)
    
    if len(next_fre_item_sets) < 1:
        return None
    else:
        return next_fre_item_sets

In [4]:
#create one value freq item sets
def get_one_value_freq_set(data,min_sup):
    data_num = len(data)
    least_freq = min_sup * data_num
    one_value_freq_set = {}
    item_set_len = 1
    
    for i in data:
        for j in i:
            if j not in one_value_freq_set.keys():
               one_value_freq_set[j] = 1
            else:
               one_value_freq_set[j] += 1
   
    one_value_freq_sets = least_freq_check(one_value_freq_set,least_freq)
    return one_value_freq_sets,least_freq,item_set_len


In [5]:
#create freq item sets from row dataset
def get_fre_item_sets(data, min_sup):
    one_value_freq_set,least_freq,item_set_len = get_one_value_freq_set(data,min_sup)
    freq_sets = [one_value_freq_set]
    continue_run = True
    
    item_set_len += 1
    while continue_run:
        if len(freq_sets[item_set_len - 2]) >= 2:
            next_freq_item_set = get_next_fre_item_set(data, freq_sets[item_set_len - 2], item_set_len,least_freq)
            item_set_len += 1
            if next_freq_item_set != None:
                freq_sets.append(next_freq_item_set)
            else:
                continue_run = False
        else:
            continue_run = False
    return freq_sets


In [6]:
#calculate lift and confidence and compare with min_sup,min_lift
def calculate_reference(fre_item_sets, subset, fre_item,min_con,min_lift,length):
    def renew_key(fre_item_sets,set1,leng,sub):
        key = None
        if len(sub) == 0:
            return None
        elif len(sub) == 1:
            key = set1[0]
        else:
            keys = list(fre_item_sets[leng - 1].keys())
            for item in keys:
                if set(set1) == set(item):
                    key = item
                    break
        return key
        
    sub_key = renew_key(fre_item_sets,subset,len(subset),subset)
    
    leftset = [i for i in list(fre_item)]

    for x in subset:
        leftset.remove(x)
        
    left_key = renew_key(fre_item_sets,leftset,len(fre_item) - len(subset),leftset)
    
    freq_key = None
    freq_keys = list(fre_item_sets[len(fre_item) - 1].keys())
    for item in freq_keys:
        if set(fre_item) == set(item):
            freq_key = item
            break

    confidence = round(fre_item_sets[len(fre_item) - 1][freq_key] / fre_item_sets[len(subset) - 1][sub_key],2)
    lift = round(confidence/ (fre_item_sets[(len(fre_item) - len(subset)) - 1][left_key] / length),2)
    if lift >= min_lift and confidence >= min_con:
        return [lift,confidence]
    return None

In [7]:
#create subsets for a list
def subsets(itemset):
    result = [[]]
    for i in range(len(itemset)):
        for j in range(len(result)):
            result.append(result[j]+[itemset[i]])
    for set in result:
        if len(set) == 0 or len(set) == len(itemset):
            result.remove(set)
    return result

In [8]:
#combie 2 dict keys into one set
def comb_ele(ele1,ele2):
    set_1 = set()
    set_2 = set()
    if type(ele1) == str:
        set_1.add(ele1)
    else:
        set_1 = set(list(ele1))
    if type(ele2) == str:
        set_2.add(ele2)
    else:
        set_2 = set(list(ele2))
    set_1.update(set_2)
    return set_1


In [9]:
#main function to generate association rules
def get_association_rules(fre_item_sets,min_lift, min_con,min_sup,length):
    association_rules = []
    for fre_item_set in fre_item_sets[1:]:
        for fre_item in list(fre_item_set.keys()):
            support = round(fre_item_set[fre_item]/length,2)
            lis = []
            all_subsets = subsets(fre_item)
            for s1 in range(len(all_subsets) - 1):
                for s2 in range(s1 + 1, len(all_subsets)):
                    subset1 = all_subsets[s1]
                    subset2 = all_subsets[s2]
                    if len(subset1) + len(subset2) == len(fre_item) and len(set(subset1) & set(subset2)) == 0:
                        reference = calculate_reference( fre_item_sets,subset1, fre_item,min_con,min_lift,length)
                        if reference != None:
                            lift = reference[0]
                            confidence = reference[1]
                            tmp = [subset1,subset2,lift,confidence,support]
                            lis.append(tmp)
                        reference = calculate_reference( fre_item_sets,subset2, fre_item,min_con,min_lift,length)
                        if reference != None:
                            lift = reference[0]
                            confidence = reference[1]
                            tmp = [subset2,subset1,lift,confidence,support]
                            lis.append(tmp)
            if len(lis) > 0:
                association_rules.append(lis)
    sort_print(association_rules)
    return association_rules


In [10]:
#sort output list to reliable format
def sort_print(data):
    temp_data_list = []
    for i in data:
        for j in i:
            temp_data_list.append(j)
    temp_data_list.sort(key=lambda x: int(len(x[0])) )
    temp_data_list.sort(key=lambda x: (x[2],x[3],x[4]) ,reverse = True )
    for ele in temp_data_list:
        temp = [ele[2],ele[3],ele[4]]
        print(*ele[0],sep = ",",end = '')
        print(' -> ',end = '')
        print(*ele[1],sep = ",",end = '')
        print("      ",end = '')
        print(*temp,sep = ",")
    return temp_data_list

In [11]:
def Apriori(data_set, min_sup, min_con, min_lift):
    fre_item_sets = get_fre_item_sets(data_set, min_sup)
    association_rules = get_association_rules(fre_item_sets,min_lift, min_con,min_sup,len(data_set))

In [13]:
min_sup = 0.25
min_con = 0.8
min_lift = 0
#data_set = loadData('supermarket.csv')
data_set = [['A', 'B', 'C', 'D', 'F', 'G', 'H'], ['A', 'B', 'C', 'D'], ['A', 'B', 'C', 'D'], ['A', 'B', 'D'], ['B', 'C', 'E']]
Apriori(data_set, min_sup, min_con, min_lift)
#the output arguments from left to right are lift,confidence,support

A -> D      1.25,1.0,0.8
D -> A      1.25,1.0,0.8
A -> B,D      1.25,1.0,0.8
D -> A,B      1.25,1.0,0.8
B,D -> A      1.25,1.0,0.8
A,B -> D      1.25,1.0,0.8
C,D -> A      1.25,1.0,0.6
A,C -> D      1.25,1.0,0.6
A,C -> B,D      1.25,1.0,0.6
C,D -> A,B      1.25,1.0,0.6
C,B,D -> A      1.25,1.0,0.6
A,C,B -> D      1.25,1.0,0.6
A -> B      1.0,1.0,0.8
C -> B      1.0,1.0,0.8
D -> B      1.0,1.0,0.8
A,D -> B      1.0,1.0,0.8
A,C -> B      1.0,1.0,0.6
C,D -> B      1.0,1.0,0.6
A,C,D -> B      1.0,1.0,0.6
B -> A      1.0,0.8,0.8
B -> C      1.0,0.8,0.8
B -> D      1.0,0.8,0.8
B -> A,D      1.0,0.8,0.8
